In [1]:
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import re

In [2]:
df = pd.read_csv('body_users.csv')

In [3]:
df.head()

,id,original_client_user_id,sex_type,age_segment,Канал привлечения,acquisition_channel_lvl_2,acquisition_channel_lvl_3
0,5981029,7788617545,М,55 лет +,Рефералка Sales,Рефералка Sales,Рефералка Sales
1,9084314,8592491475,М,35-44 лет,Банк,Банк,Банк
2,1461670,6606290372,Ж,35-44 лет,Рефералка Sales,Рефералка Sales,Рефералка Sales
3,5891305,7169207061,Ж,55 лет +,Маркетинг KZ,Реферальные,Реферрер не определен
4,7353391,2506521164,М,45-54 лет,Белиз,Белиз,Белиз


In [4]:
events_files = glob('users_events*.csv')
events = pd.concat((pd.read_csv(f) for f in events_files))
events['date_mod'] = pd.to_datetime(events['date_mod'])
events.sort_values('date_mod', inplace=True)

In [5]:
events

,id,user_id,name,date_mod
0,778739486,9821728,Change in management,2025-01-01 00:00:07.008
1,778739492,5970559,Trade result successful,2025-01-01 00:00:08.018
2,778739497,1162986,Trade result successful,2025-01-01 00:00:08.396
3,778739498,8835246,User logged in,2025-01-01 00:00:09.433
4,778739499,8835246,Authentication completed,2025-01-01 00:00:09.440
...,...,...,...,...
3147531,1113334970,4153352,Authentication completed,2025-06-30 23:59:55.320
2613299,1113334976,3039855,User logged in,2025-06-30 23:59:57.692
4658443,1113334977,3039855,Authentication completed,2025-06-30 23:59:57.701
4558822,1113334980,12585206,Viewing mode of reception adjusted,2025-06-30 23:59:59.033


In [6]:
events.groupby("name").agg(
    rows=("name", "size"),             # total rows per event
    unique_accounts=("user_id", "nunique"),  # unique account ids per event
    unique_event_ids=("id", "nunique"),      # optional: unique event ids (drop if no 'id' col)
).sort_values(["unique_accounts", "rows"], ascending=False)


,rows,unique_accounts,unique_event_ids
name,,,
Change in management,1724443,1657276,1724443
Reception service updated,2214392,1297917,2214392
User logged in,6933244,485062,6933244
Dormant Account Status Updated,472573,423507,472573
Authentication completed,6704807,306946,6704807
...,...,...,...
Subordination entry substituted,10,1,10
File reinstated,1,1,1
Fourth referral account registered,1,1,1


In [7]:
ACC_DF   = "id"           # account id in df
AGE_COL  = "age_segment"  # age segment
ACC_EVT  = "user_id"      # account id in events
EVT_COL  = "name"         # event name in events (rename if your file uses 'Name')
if EVT_COL not in events.columns and "Name" in events.columns:
    events = events.rename(columns={"Name": EVT_COL})

In [8]:
# Cell 3
steps_raw = [
    "Unable to create account",
    "Account successfully created for user",
    "Account setup completed successfully",
    "Mobile number was linked to the account",
    "User profile status changed",
    "Account set up through a mobile device",
    "Mobile number verification completed",
    "Mobile number verification completed",  # duplicate in your list
    "Account registered under a personal name",
    "Initiated account registration",
    "Email verification",
    "Personal data submitted",
    "User session established",
    "Account documents submitted",
    "Fourth stage finalized",
    "Fifth stage finalized",
    "User profile verified",
    "Chosen a pricing plan for the accoun",  # keep exactly as you have it
    "Account opened successfully",
    "Switched to live environment",
    "Account created for a business or organization",
    "Account contract signed",
    "Referral enrollment finalized",
    "User role updated",
    "IPO participation option selected",
]

# Normalize ONLY whitespace; do not change spelling
def norm_event_label(s: str) -> str:
    return re.sub(r"\s+", " ", str(s).strip())

# Deduplicate while preserving order
seen, STEPS = set(), []
for s in steps_raw:
    s = norm_event_label(s)
    if s not in seen:
        seen.add(s)
        STEPS.append(s)

# Also normalize event names in the events data the same way (whitespace only)
events[EVT_COL] = events[EVT_COL].astype(str).map(norm_event_label)


In [9]:
# Cell 4
age_key = df[[ACC_DF, AGE_COL]].drop_duplicates()

ev = (
    events[[ACC_EVT, EVT_COL]]
      .merge(age_key, left_on=ACC_EVT, right_on=ACC_DF, how="left")
      .rename(columns={ACC_EVT: "account_id", EVT_COL: "step"})
)

# Label missing ages (if some accounts aren’t in df)
ev[AGE_COL] = ev[AGE_COL].fillna("Не указано клиентом")

# Keep only your fixed steps
ev = ev[ev["step"].isin(STEPS)].copy()


In [10]:
# Cell 5
# Count UNIQUE account_ids per (step, age_segment)
dedup = ev.drop_duplicates(subset=["account_id", "step"])

step_age_counts = (
    dedup.groupby(["step", AGE_COL])["account_id"]
         .nunique()
         .unstack(AGE_COL)
         .reindex(STEPS)          # enforce your fixed row order
         .fillna(0)
         .astype(int)
)

# Optional: order age columns nicely (use the categories you showed)
age_order = ["<18 лет","18-24 лет","25-34 лет","35-44 лет","45-54 лет","55 лет +","Не указано клиентом"]
step_age_counts = step_age_counts.reindex(columns=[a for a in age_order if a in step_age_counts.columns])

step_age_counts


age_segment,18-24 лет,25-34 лет,35-44 лет,45-54 лет,55 лет +,Не указано клиентом
step,,,,,,
Unable to create account,1573,2088,1991,898,427,5743
Account successfully created for user,11995,13964,11569,5180,2404,190165
Account setup completed successfully,11995,13964,11569,5180,2404,190165
Mobile number was linked to the account,387,875,1233,764,522,4721
User profile status changed,13140,15198,12566,5555,2547,56893
Account set up through a mobile device,12355,14336,11877,5318,2452,56617
Mobile number verification completed,318,726,1043,675,446,3985
Account registered under a personal name,356,787,1118,693,458,2603
Initiated account registration,11680,13237,10506,4482,1938,187802


In [11]:
# Merge age onto events first (df.id ↔ events.user_id)
sex_key = df[["id", "sex_type"]].drop_duplicates()
ev = (events[["user_id", "name", "id"]]
      .merge(sex_key, left_on="user_id", right_on="id", how="left", suffixes=("_evt","_df"))
      .rename(columns={"user_id":"account_id", "name":"step"}))

# Overall per event (rows + unique accounts + unique event ids)
overall = ev.groupby("step").agg(
    rows=("step", "size"),
    unique_accounts=("account_id", "nunique"),
    unique_event_ids=("id_evt", "nunique") if "id_evt" in ev.columns else ("account_id", "count")
).sort_values(["unique_accounts","rows"], ascending=False)


In [12]:
import unicodedata

# 0) If sex_type comes from df (by account), make sure it's merged onto ev first:
# ev = events.merge(df[['id','sex_type']], left_on='user_id', right_on='id', how='left')...

# 1) Normalize sex_type
def norm_sex(x):
    if pd.isna(x):
        return "Не указано клиентом"
    s = str(x).strip()
    s = unicodedata.normalize("NFKC", s)  # unify Unicode forms
    u = s.upper()
    # Map common variants to your three buckets
    if u in {"M", "М", "MALE", "МУЖ", "МУЖЧИНА"}:
        return "М"
    if u in {"F", "Ж", "FEMALE", "ЖЕН", "ЖЕНЩИНА"}:
        return "Ж"
    if u in {"", "NAN", "NONE", "NULL"}:
        return "Не указано клиентом"
    return s  # already one of your labels

ev["sex_type"] = ev["sex_type"].map(norm_sex)

# 2) Dedup accounts per step
dedup = ev.drop_duplicates(subset=["account_id", "step"])

# 3) Build step × sex table (rows = steps, cols = sex)
sex_order = ["М", "Ж", "Не указано клиентом"]  # NOTE: Cyrillic М/Ж
step_sex_counts = (
    dedup.groupby(["step", "sex_type"])["account_id"]
         .nunique()
         .unstack("sex_type")
         .reindex(STEPS)                    # enforce your step row order
         .reindex(columns=sex_order)        # force all three columns to exist
         .fillna(0)
         .astype(int)
)

print(step_sex_counts)

sex_type                                            М      Ж  \
step                                                           
Unable to create account                         4224   2749   
Account successfully created for user           26800  18312   
Account setup completed successfully            26800  18312   
Mobile number was linked to the account          2193   1585   
User profile status changed                     29409  19592   
Account set up through a mobile device          27578  18760   
Mobile number verification completed             1817   1391   
Account registered under a personal name         1947   1464   
Initiated account registration                  24952  16891   
Email verification                                 38     24   
Personal data submitted                             0      0   
User session established                        47423  31113   
Account documents submitted                         0      0   
Fourth stage finalized                  

In [19]:
step_sex_counts

sex_type,М,Ж,Не указано клиентом
step,,,
Unable to create account,4224,2749,5747
Account successfully created for user,26800,18312,190165
Account setup completed successfully,26800,18312,190165
Mobile number was linked to the account,2193,1585,4724
User profile status changed,29409,19592,56898
Account set up through a mobile device,27578,18760,56617
Mobile number verification completed,1817,1391,3985
Account registered under a personal name,1947,1464,2604
Initiated account registration,24952,16891,187802


In [14]:
#now lets save age and sex
step_age_counts.to_csv('/content/drive/MyDrive/datathon/step_age_counts.csv')
step_sex_counts.to_csv('/content/drive/MyDrive/datathon/step_sex_counts.csv')

In [13]:
# Cell 2
# df.csv     -> has: id (account id), channel columns
# events.csv -> has: user_id (account id), name (event name)

ACC_DF   = "id"                          # account id in df
ACC_EVT  = "user_id"                     # account id in events
EVT_COL  = "name"                        # event name in events

# Channel columns (edit if your headers differ)
CH1 = "Канал привлечения"
CH2 = "acquisition_channel_lvl_2"
CH3 = "acquisition_channel_lvl_3"

# If events uses 'Name' instead of 'name'
if EVT_COL not in events.columns and "Name" in events.columns:
    events = events.rename(columns={"Name": EVT_COL})


In [14]:
# Cell 3
def norm_txt(s: str) -> str:
    return re.sub(r"\s+", " ", str(s).strip())

# unify keys to strings
df[ACC_DF]      = df[ACC_DF].astype(str).str.strip()
events[ACC_EVT] = events[ACC_EVT].astype(str).str.strip()

# normalize event names and channels (whitespace only; no renaming)
events[EVT_COL] = events[EVT_COL].astype(str).map(norm_txt)
for col in [CH1, CH2, CH3]:
    if col in df.columns:
        df[col] = df[col].astype(str).map(norm_txt)


In [15]:
# Cell 4
channel_cols = [c for c in [CH1, CH2, CH3] if c in df.columns]
age_key = df[[ACC_DF] + channel_cols].drop_duplicates()

ev = (events[[ACC_EVT, EVT_COL]]
      .merge(age_key, left_on=ACC_EVT, right_on=ACC_DF, how="left")
      .rename(columns={ACC_EVT: "account_id", EVT_COL: "step"}))

# optional: label missing channels consistently
for col in channel_cols:
    ev[col] = ev[col].fillna("Не указано")


In [16]:
# Cell 6
overall = {}
for col in channel_cols:
    overall[col] = (ev.groupby(col)["account_id"]
                      .nunique()
                      .sort_values(ascending=False)
                      .rename("unique_accounts"))
overall  # dict of Series: overall[CH1], overall[CH2], overall[CH3]


{'Канал привлечения': Канал привлечения
 Не указано         1790684
 Органика            199159
 Маркетинг KZ        133433
 Рефералка Sales      98688
 Name: unique_accounts, dtype: int64,
 'acquisition_channel_lvl_2': acquisition_channel_lvl_2
 Не указано                  1790684
 Органика                     199159
 Рефералка Sales               98688
 Рекламные Кабинеты            59575
 Лендинг                       51840
 Реферальные                   11348
 Блоггеры                       5643
 Не определен Маркетингом       4281
 Не удалось определить           593
 Рассылки                         52
 старая ссылка                    35
 Ивенты                           34
 bytedanceglobal_int              32
 Name: unique_accounts, dtype: int64,
 'acquisition_channel_lvl_3': acquisition_channel_lvl_3
 Не указано           1790684
 Органика              199159
 Рефералка Sales        98688
 Google                 46304
 ffinkz                 40800
                       ...   

In [17]:
# Cell 7
# de-dup so each account counts once per (step, channel)
dedup = ev.drop_duplicates(subset=["account_id", "step"])

step_channel_tables = {}
for col in channel_cols:
    tbl = (dedup.groupby(["step", col])["account_id"]
                .nunique()
                .unstack(col)
                .fillna(0).astype(int))
    # if you have a fixed STEPS order, enforce it:
    # if 'STEPS' in globals():
    #     tbl = tbl.reindex(STEPS)
    step_channel_tables[col] = tbl

# Examples:
# step_channel_tables[CH1]  # rows=steps, cols=Канал привлечения
# step_channel_tables[CH2]  # rows=steps, cols=acquisition_channel_lvl_2
# step_channel_tables[CH3]  # rows=steps, cols=acquisition_channel_lvl_3


In [23]:
step_channel_tables[CH1].reset_index().to_csv('channels1.csv')
step_channel_tables[CH2].reset_index().to_csv('channels2.csv')
step_channel_tables[CH3].reset_index().to_csv('channels3.csv')


In [18]:
# Cell 8
def topn_step_channel(ev_tbl: pd.DataFrame, top_n: int = 12):
    """
    Keep Top-N columns by total, sum the rest into 'Other'.
    ev_tbl: step × channel pivot (integers)
    """
    totals = ev_tbl.sum(axis=0).sort_values(ascending=False)
    keep = list(totals.head(top_n).index)
    rest = [c for c in ev_tbl.columns if c not in keep]
    out = ev_tbl[keep].copy()
    if rest:
        out["Other"] = ev_tbl[rest].sum(axis=1)
    return out

# Example: compress lvl_3 to Top-15 + Other
top15_lvl3 = topn_step_channel(step_channel_tables.get(CH3, pd.DataFrame()), top_n=15)
top15_lvl3


acquisition_channel_lvl_3,Не указано,Органика,Рефералка Sales,Google,ffinkz,TikTok,global,Не определен Маркетингом,tradernet.global,Лендинг не определен,Реферрер не определен,Apple Search,Yandex,Meta,fbroker,Other
step,,,,,,,,,,,,,,,,
Account contract signed,473,6,74,0,0,0,1,2,0,0,0,0,0,0,0,1
Account created for a business or organization,56,5,2,1,2,0,1,0,1,0,0,0,0,0,0,0
Account creation unsuccessful,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Account documents submitted,522,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Account opened successfully,249,60576,6733,15179,2608,3830,1468,1122,280,1650,143,276,294,442,465,661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Withdraw from Deposit,3100,4362,4905,471,879,109,133,197,181,38,125,66,25,11,10,144
Withdrawal Successful,3613,12403,10226,1216,2634,208,222,346,520,80,301,226,75,21,15,467
apply_instrument_limit,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
